In [97]:
### WILL BE NEEDED LATER

## Split data into training and test sets
# train_data, test_data = train_test_split(data, test_size = 0.2, random_state = 42)
# X_train = train_data[independent_vars]
# y_train = train_data[dependent_var]

## Evaluate the model
# X_test = test_data[independent_vars]
# y_test = test_data[dependent_var]
# X_test = sm.add_constant(X_test)
# y_pred = model.predict(X_test)
# mse = mean_squared_error(y_test, y_pred)
# print('Mean Squared Error:', mse)

## Check multicollinearity using VIF
# vif_data = pd.DataFrame()
# vif_data['feature'] = X_train.columns
# vif_data['VIF'] = [variance_inflation_factor(X_train.values, i) for i in range(X_train.shape[1])]
# print(vif_data)

In [98]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor
import statsmodels.formula.api as smf
import matplotlib.pyplot as plt
from sklearn.model_selection import LeaveOneOut, train_test_split
from sklearn.metrics import mean_squared_error
from scipy.stats import f

In [99]:
# Step 1: Data Preparation
# Load dataset
data = pd.read_csv('biele_WM_new.csv')

In [100]:
# Select relevant variables and update original dataset to data
columns = ["mietekalt", "qm_Preis", "wohnflaeche", "alter", "objektzustand", "zimmeranzahl", 
           "balkon", "einbaukueche", "keller", "ausstattung_kat", "aufzug", "gaestewc", 
           "garten", "heizungsart", "barrierefrei", "Einstellungsmonat"]
data = data[columns]

In [101]:
# Rename variables for clarity
data.rename(columns = {'mietekalt': 'netrent', 
                     'qm_Preis': 'rent_per_sqm', 
                     'wohnflaeche': 'area', 
                     'alter': 'age', 
                     'objektzustand': 'condition_cat', 
                     'zimmeranzahl': 'rooms', 
                     'balkon': 'balcony', 
                     'einbaukueche': 'kitchen', 
                     'keller': 'basement', 
                     'ausstattung_kat': 'appointments_cat', 
                     'aufzug': 'lift', 
                     'gaestewc': 'guesttoilet', 
                     'garten': 'garden', 
                     'heizungsart': 'heating_cat', 
                     'barrierefrei': 'barrierfree', 
                     'Einstellungsmonat': 'month'}, inplace= True)

In [102]:
# Drop rows with missing values
data = data.dropna()

In [103]:
# Encode categorical variables
data = pd.get_dummies(data,
                      columns = ['condition_cat', 'appointments_cat', 'heating_cat', 'month'],
                      drop_first = True, dtype = int)

In [104]:
# Define dependent and independent variables
dependent_var = 'netrent'
independent_vars = [col for col in data.columns if col != dependent_var]

# MODEL WITH statsmodels

In [105]:
# Fit the linear regression model
X = data[independent_vars]
y = data[dependent_var]
X = sm.add_constant(X)
model = sm.OLS(y, X).fit()

In [106]:
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                netrent   R-squared:                       0.959
Model:                            OLS   Adj. R-squared:                  0.959
Method:                 Least Squares   F-statistic:                     1311.
Date:                Fri, 17 Jan 2025   Prob (F-statistic):               0.00
Time:                        16:57:57   Log-Likelihood:                -12554.
No. Observations:                2371   AIC:                         2.519e+04
Df Residuals:                    2328   BIC:                         2.544e+04
Df Model:                          42                                         
Covariance Type:            nonrobust                                         
                         coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------
const               -523.9484     17

# MODEL BY HAND

In [107]:
# Extract values of dependent and independent variables
X_val = data[independent_vars].values
y_val = data[dependent_var].values

In [108]:
# Add intercept
intercept = np.ones((X_val.shape[0], 1))             # Create a column of ones
X_with_intercept = np.hstack((intercept, X_val))     # Combine intercept with X

In [109]:
# X_1: Design matrix (including intercept column)
# y_1: Dependent variable (netrent)
X_1 = np.array(X_with_intercept)
y_1 = np.array(y_val)

# Estimate beta_hat
beta_hat_1 = np.linalg.inv(X_1.T @ X_1) @ X_1.T @ y_1

# Compute the hat matrix (H)
H_1 = X_1 @ np.linalg.inv(X_1.T @ X_1) @ X_1.T

# Residuals (epsilon_hat)
epsilon_hat_1 = (np.eye(len(y_1)) - H_1) @ y_1

# Fitted values (y_hat)
y_hat_1 = H_1 @ y_1

# Estimate sigma^2 (residual variance)
n, k = X_1.shape
sigma2_hat_1 = (1 / (n - k)) * (epsilon_hat_1.T @ epsilon_hat_1)

# Covariance matrix of beta_hat
Cov_beta_hat_1 = sigma2_hat_1 * np.linalg.inv(X_1.T @ X_1)

# Standard errors of beta_hat
sd_beta_hat_1 = np.sqrt(np.diag(Cov_beta_hat_1))

In [110]:
# Print results
print("Beta_hat:", beta_hat_1)
print("Residual variance (sigma^2):", sigma2_hat_1)
print("Covariance matrix of beta_hat:\n", Cov_beta_hat_1)
print("Standard errors of beta_hat:", sd_beta_hat_1)

Beta_hat: [-5.23948384e+02  6.10658948e+01  8.59321431e+00  5.53620883e-02
 -2.54468096e+00 -8.06528052e-01 -2.70222358e+00  7.98397364e+00
  1.26679156e+01 -2.41513406e+00 -6.90621988e+00 -5.81377010e-01
 -3.72479491e+01 -1.85609140e+01 -4.83964599e+01 -3.20319804e+01
 -3.40961045e+01 -3.61420108e+01 -2.68937018e+01 -4.35903126e+00
  4.50005655e+00 -8.95673158e+00  2.14116961e+01  2.13354685e+01
  2.63116797e+01  1.79557514e+01  6.29993033e+01  1.34671687e+01
  2.01176338e+01  7.45775517e+01  9.09234023e+00  1.64397137e+01
  5.78458433e+00  4.18994603e+00  1.90566502e+00  5.52617826e+00
  2.91243946e-01  1.06345318e+00 -5.51686719e+00  3.05773568e+00
 -1.99978245e-01 -1.95134315e+00 -2.52185488e+00]
Residual variance (sigma^2): 2368.123936743671
Covariance matrix of beta_hat:
 [[ 3.15628328e+02 -4.43861665e+00 -8.02369551e-02 ... -1.33214374e+01
  -9.85959626e+00 -8.62537925e+00]
 [-4.43861665e+00  4.52350495e-01  5.24957574e-03 ... -1.24591685e-01
  -1.57741243e-01 -1.69724940e-01]
 

In [111]:
# Comparison of coefficients and standard deviations
coeff_comparison = np.vstack([model.params, beta_hat_1])
stddev_comparison = np.vstack([model.bse, sd_beta_hat_1])

In [112]:
# Display results
print("Comparison of Coefficients:")
print(pd.DataFrame(coeff_comparison, index=["Statsmodels", "By-Hand"], 
                   columns=['intercept'] + independent_vars))
print("\nComparison of Standard Deviations:")
print(pd.DataFrame(stddev_comparison, index=["Statsmodels", "By-Hand"], 
                   columns=['intercept'] + independent_vars))

Comparison of Coefficients:
              intercept  rent_per_sqm      area       age     rooms   balcony  \
Statsmodels -523.948384     61.065895  8.593214  0.055362 -2.544681 -0.806528   
By-Hand     -523.948384     61.065895  8.593214  0.055362 -2.544681 -0.806528   

              kitchen  basement       lift  guesttoilet  ...   month_3  \
Statsmodels -2.702224  7.983974  12.667916    -2.415134  ...  4.189946   
By-Hand     -2.702224  7.983974  12.667916    -2.415134  ...  4.189946   

              month_4   month_5   month_6   month_7   month_8   month_9  \
Statsmodels  1.905665  5.526178  0.291244  1.063453 -5.516867  3.057736   
By-Hand      1.905665  5.526178  0.291244  1.063453 -5.516867  3.057736   

             month_10  month_11  month_12  
Statsmodels -0.199978 -1.951343 -2.521855  
By-Hand     -0.199978 -1.951343 -2.521855  

[2 rows x 43 columns]

Comparison of Standard Deviations:
             intercept  rent_per_sqm      area       age     rooms  balcony  \
Statsmode

# HYPOTHESIS TESTING

In [113]:
# List of variables to exclude
exclude_vars = ['age', 'rooms', 'balcony', 'kitchen', 'guesttoilet', 'barrierfree']

# Restrict independent_vars by excluding the specified variables
independent_vars_H0 = [col for col in independent_vars if col not in exclude_vars]

In [114]:
# Restricted independent variables
X_H0 = data[independent_vars_H0]

# Add intercept to X_H0
X_H0 = sm.add_constant(X_H0)

In [115]:
# Fit the reduced model
model_H0 = sm.OLS(y, X_H0).fit()

In [116]:
# Print results
print(model_H0.summary())

                            OLS Regression Results                            
Dep. Variable:                netrent   R-squared:                       0.959
Model:                            OLS   Adj. R-squared:                  0.959
Method:                 Least Squares   F-statistic:                     1530.
Date:                Fri, 17 Jan 2025   Prob (F-statistic):               0.00
Time:                        16:57:57   Log-Likelihood:                -12557.
No. Observations:                2371   AIC:                         2.519e+04
Df Residuals:                    2334   BIC:                         2.540e+04
Df Model:                          36                                         
Covariance Type:            nonrobust                                         
                         coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------
const               -524.6984     17

##### PERFORM F-TEST

In [117]:
SSE_full = np.sum(model.resid ** 2)             # Residual Sum of Squares for full model
SSE_H0 = np.sum(model_H0.resid ** 2)            # Residual Sum of Squares for reduced model

n = len(y)                 # Number of observations
p_full = X.shape[1]        # Number of parameters in full model
p_H0 = X_H0.shape[1]       # Number of parameters in reduced model
r = p_full - p_H0          # Number of restrictions

In [118]:
# Calculate F-statistic
F_H0 = ((SSE_H0 - SSE_full) / r) / (SSE_full / (n - p_full))
print(f"F-statistic: {F_H0}")

# f_statistics = model.compare_f_test(model_H0)

F-statistic: 0.9905492751696735


In [119]:
# Calculate critical value
alpha = 0.05
F_critical = f.ppf(1 - alpha, dfn=r, dfd=n - p_full)
print(f"Critical value (alpha={alpha}): {F_critical}")

Critical value (alpha=0.05): 2.1024737192532603


In [120]:
# Calculate p-value
p_value = 1 - f.cdf(F_H0, dfn=r, dfd=n - p_full)
print(f"P-value: {p_value}")

P-value: 0.42985218568727157


In [121]:
# Interpretation of results
if p_value < alpha:
    print("Reject the null hypothesis: The variables are jointly significantly different than zero.")
else:
    print("Fail to reject the null hypothesis: The variables are jointly not significantly different than zero.")

Fail to reject the null hypothesis: The variables are jointly not significantly different than zero.


# MODEL SELECTION

In [122]:
# Compare adjusted R-squared
adj_rsq_model = model.rsquared_adj
adj_rsq_model_H0 = model_H0.rsquared_adj

print("Adjusted R-squared:")
print(f"Full model: {adj_rsq_model}")
print(f"Reduced model: {adj_rsq_model_H0}")

Adjusted R-squared:
Full model: 0.9587085934428334
Reduced model: 0.9587095966143533


In [123]:
# Leverage values (hat matrix diagonal elements)
influence_model = model.get_influence()
h_ii_1 = influence_model.hat_matrix_diag

influence_model_H0 = model_H0.get_influence()
h_ii_2 = influence_model_H0.hat_matrix_diag

In [124]:
# Compute leave-one-out cross-validation error (LOOCV)
y_actual = model.model.endog  # Dependent variable values

loocv_model = np.mean(((y_actual - model.fittedvalues) / (1 - h_ii_1)) ** 2)
loocv_model_H0 = np.mean(((y_actual - model_H0.fittedvalues) / (1 - h_ii_2)) ** 2)

print("\nLOOCV Error:")
print(f"Full model: {loocv_model}")
print(f"Reduced model: {loocv_model_H0}")


LOOCV Error:
Full model: 21128.229153659093
Reduced model: 4346.611089567004


In [125]:
# THE RESTRICTED MODEL, model_H0, EXPLAINS THE DATA BETTER THAN THE FULL MODEL, model.
# model_H0 IS NEW CURRENT MODEL

# PROBLEM ANALYSIS

In [126]:
# Identify problematic observations where leverage = 1
high_leverage_1 = np.where(h_ii_1 == 1)[0]
high_leverage_2 = np.where(h_ii_2 == 1)[0]

print("Observations with h_ii == 1 in full model:", high_leverage_1)
print("Observations with h_ii == 1 in reduced model:", high_leverage_2)

Observations with h_ii == 1 in full model: []
Observations with h_ii == 1 in reduced model: []


In [127]:
# NO PROBLEM AS 1-h_i_i WILL NEVER BE 0 ANYWHERE IN DIAGONAL OF THE HAT MATRIX

In [164]:
# Reload the original dataset
data = pd.read_csv('biele_WM_new.csv')
columns = ["mietekalt", "qm_Preis", "wohnflaeche", "alter", "objektzustand", "zimmeranzahl", "balkon", "einbaukueche", "keller", "ausstattung_kat", "aufzug", "gaestewc", "garten", "heizungsart", "barrierefrei", "Einstellungsmonat"]
data = data[columns]
data.rename(columns = {'mietekalt': 'netrent', 
                     'qm_Preis': 'rent_per_sqm', 
                     'wohnflaeche': 'area', 
                     'alter': 'age', 
                     'objektzustand': 'condition_cat', 
                     'zimmeranzahl': 'rooms', 
                     'balkon': 'balcony', 
                     'einbaukueche': 'kitchen', 
                     'keller': 'basement', 
                     'ausstattung_kat': 'appointments_cat', 
                     'aufzug': 'lift', 
                     'gaestewc': 'guesttoilet', 
                     'garten': 'garden', 
                     'heizungsart': 'heating_cat', 
                     'barrierefrei': 'barrierfree', 
                     'Einstellungsmonat': 'month'}, inplace= True)
data = data.dropna()

# Check categorical variable distribution
print("Condition category summary:\n", data['condition_cat'].value_counts())
print("Appointments category summary:\n", data['appointments_cat'].value_counts())
print("Heating category summary:\n", data['heating_cat'].value_counts())
print("Month summary:\n", data['month'].value_counts())

Condition category summary:
 condition_cat
7.0    1198
6.0     277
5.0     243
3.0     240
1.0     185
2.0     135
4.0      78
8.0      15
Name: count, dtype: int64
Appointments category summary:
 appointments_cat
0    875
1    768
2    728
Name: count, dtype: int64
Heating category summary:
 heating_cat
13.0    1150
3.0      314
4.0      300
6.0      285
5.0      208
10.0      51
12.0      35
1.0       11
2.0        7
8.0        5
7.0        4
11.0       1
Name: count, dtype: int64
Month summary:
 month
7     334
1     217
11    204
9     200
10    198
12    194
3     192
8     191
2     186
6     182
4     168
5     105
Name: count, dtype: int64


In [165]:
# Encode categorical variables
data = pd.get_dummies(data,
                      columns = ['condition_cat', 'heating_cat', 'appointments_cat', 'month'],
                      drop_first = True, dtype = int)

In [166]:
# Define a new restricted model excluding the non-significant heating categories from model_H0
independent_vars_H01 = [
    "area", "basement", "lift", "garden", "heating_cat_7.0"
]+ [col for col in data.columns if col.startswith(("condition_cat_", "appointments_cat_", "month_"))]  

X_H01 = sm.add_constant(data[independent_vars_H01])
model_H01 = sm.OLS(data[dependent_var], X_H01).fit()

In [148]:
# Print results
print(model_H01.summary())

                            OLS Regression Results                            
Dep. Variable:                netrent   R-squared:                       0.800
Model:                            OLS   Adj. R-squared:                  0.798
Method:                 Least Squares   F-statistic:                     374.4
Date:                Fri, 17 Jan 2025   Prob (F-statistic):               0.00
Time:                        17:31:18   Log-Likelihood:                -14447.
No. Observations:                2371   AIC:                         2.895e+04
Df Residuals:                    2345   BIC:                         2.910e+04
Df Model:                          25                                         
Covariance Type:            nonrobust                                         
                         coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------
const                138.5604     14

##### PERFORM F-TEST

In [149]:
SSE = np.sum(model.resid**2)
SSE_H01 = np.sum(model_H01.resid**2)
n = len(model.fittedvalues)
p = model.df_model + 1                    # Number of parameters including intercept
r = model.df_model - model_H01.df_model   # Difference in model complexity

In [150]:
# Compute F-statistic
F_H01 = ((n - p) / r) * (SSE_H01 - SSE) / SSE
print(f"F-statistic: {F_H01}")

# f_statistics = model.compare_f_test(model_H01)

F-statistic: 539.4823577126677


In [151]:
# Compute critical value and p-value
F_critical = f.ppf(0.95, dfn=r, dfd=n - p)
p_value = 1 - f.cdf(F_H01, dfn=r, dfd=n - p)

print(f"Critical value at 95% confidence: {F_critical}")
print(f"P-value: {p_value}")

Critical value at 95% confidence: 1.6271598990820488
P-value: 1.1102230246251565e-16


In [152]:
if p_value < 0.05:
    print("Reject the null hypothesis: Heating categories jointly significantly influence the model.")
else:
    print("Fail to reject the null hypothesis: Heating categories have no joıntly significant influence.")

Reject the null hypothesis: Heating categories jointly significantly influence the model.


##### MODEL SELECTION

In [154]:
# Compare AIC and BIC for model selection
aic_model = model.aic
bic_model = model.bic

aic_H01 = model_H01.aic
bic_H01 = model_H01.bic

print(f"Full Model AIC: {aic_model:.2f}, BIC: {bic_model:.2f}")
print(f"Restricted Model AIC: {aic_H01:.2f}, BIC: {bic_H01:.2f}")

# Interpretation
if aic_H01 < aic_model:
    print("The model_H01 has a lower AIC and is preferred based on AIC.")
else:
    print("The model_H0 is preferred based on AIC.")

if bic_H01 < bic_model:
    print("The model_H01 has a lower BIC and is preferred based on BIC.")
else:
    print("The model_H0 is preferred based on BIC.")

Full Model AIC: 25193.53, BIC: 25441.69
Restricted Model AIC: 28946.66, BIC: 29096.70
The model_H0 is preferred based on AIC.
The model_H0 is preferred based on BIC.


In [137]:
# THE FIRST RESTRICTED MODEL, model_H0, EXPLAINS THE DATA STILL BETTER THAN THE SECOND ONE, model_H01

In [169]:
# Define a new restricted model excluding the non-significant appointment categories from model_H0
independent_vars_H02 = [
    "area", "basement", "lift", "garden",
]+ [col for col in data.columns if col.startswith(("condition_cat_", "heating_cat_", "month_"))]  

X_H02 = sm.add_constant(data[independent_vars_H02])
model_H02 = sm.OLS(data[dependent_var], X_H02).fit()

In [171]:
# Print results
print(model_H02.summary())

                            OLS Regression Results                            
Dep. Variable:                netrent   R-squared:                       0.791
Model:                            OLS   Adj. R-squared:                  0.788
Method:                 Least Squares   F-statistic:                     268.3
Date:                Fri, 17 Jan 2025   Prob (F-statistic):               0.00
Time:                        18:01:55   Log-Likelihood:                -14496.
No. Observations:                2371   AIC:                         2.906e+04
Df Residuals:                    2337   BIC:                         2.926e+04
Df Model:                          33                                         
Covariance Type:            nonrobust                                         
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
const               107.7074     36.41

In [172]:
# Compare adjusted R-squared
adj_rsq_H0 = model_H0.rsquared_adj
adj_rsq_H02 = model_H02.rsquared_adj

print(f"Adjusted R-squared for model_H0: {adj_rsq_H0:.4f}")
print(f"Adjusted R-squared for model_H02: {adj_rsq_H02:.4f}")

if adj_rsq_H02 > adj_rsq_H0:
    print("model_H02 is favored based on adjusted R-squared.")
else:
    print("model_H0 is favored based on adjusted R-squared.")

Adjusted R-squared for model_H0: 0.9587
Adjusted R-squared for model_H02: 0.7882
model_H0 is favored based on adjusted R-squared.


In [173]:
# Leave-One-Out Cross Validation (LOOCV)
h_ii_H0 = model_H0.get_influence().hat_matrix_diag
h_ii_H02 = model_H02.get_influence().hat_matrix_diag

loocv_error_H0 = np.mean(((data["netrent"] - model_H0.fittedvalues) / (1 - h_ii_H0))**2)
loocv_error_H02 = np.mean(((data["netrent"] - model_H02.fittedvalues) / (1 - h_ii_H02))**2)

print(f"LOOCV Error for model_H0: {loocv_error_H0:.4f}")
print(f"LOOCV Error for model H02: {loocv_error_H02:.4f}")

if loocv_error_H02 < loocv_error_H0:
    print("model_H02 is favored based on lower cross-validation error.")
else:
    print("model_H0 is favored based on lower cross-validation error.")

LOOCV Error for model_H0: 4346.6111
LOOCV Error for model H02: 989276.9569
model_H0 is favored based on lower cross-validation error.


In [174]:
# Compare AIC
aic_model_H0 = model_H0.aic
aic_model_H02 = model_H02.aic

print(f"AIC for model_H0: {aic_model_H0:.2f}")
print(f"AIC for model_H02: {aic_model_H02:.2f}")

if aic_model_H02 < aic_model_H0:
    print("model_H02 is favored based on AIC.")
else:
    print("model_H0 is favored based on AIC.")

# Compare BIC
bic_model_H0 = model_H0.bic
bic_model_H02 = model_H02.bic

print(f"BIC for model_H0: {bic_model_H0:.2f}")
print(f"BIC for model_H02: {bic_model_H02:.2f}")

if bic_model_H02 < bic_model_H0:
    print("model_H02 is favored based on BIC.")
else:
    print("model_H0 is favored based on BIC.")

AIC for model_H0: 25187.58
AIC for model_H02: 29060.96
model_H0 is favored based on AIC.
BIC for model_H0: 25401.11
BIC for model_H02: 29257.17
model_H0 is favored based on BIC.


In [ ]:
# model_H0 IS STILL THE FAVORITE MODEL

In [175]:
# Define a new restricted model excluding the non-significant month categories from model_H0
independent_vars_H03 = [
    "area", "basement", "lift", "garden"
]+ [col for col in data.columns if col.startswith(("condition_cat_", "appointments_cat_", "heating_cat_"))]  

X_H03 = sm.add_constant(data[independent_vars_H03])
model_H03 = sm.OLS(data[dependent_var], X_H03).fit()

In [177]:
# Print results
print(model_H03.summary())

                            OLS Regression Results                            
Dep. Variable:                netrent   R-squared:                       0.805
Model:                            OLS   Adj. R-squared:                  0.803
Method:                 Least Squares   F-statistic:                     403.3
Date:                Fri, 17 Jan 2025   Prob (F-statistic):               0.00
Time:                        18:14:13   Log-Likelihood:                -14416.
No. Observations:                2371   AIC:                         2.888e+04
Df Residuals:                    2346   BIC:                         2.903e+04
Df Model:                          24                                         
Covariance Type:            nonrobust                                         
                         coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------
const                 85.5651     34

In [178]:
# Compare adjusted R-squared
adj_rsq_H0 = model_H0.rsquared_adj
adj_rsq_H03 = model_H03.rsquared_adj

print(f"Adjusted R-squared for model_H0: {adj_rsq_H0:.4f}")
print(f"Adjusted R-squared for model_H03: {adj_rsq_H03:.4f}")

if adj_rsq_H03 > adj_rsq_H0:
    print("model_H03 is favored based on adjusted R-squared.")
else:
    print("model_H0 is favored based on adjusted R-squared.")

Adjusted R-squared for model_H0: 0.9587
Adjusted R-squared for model_H03: 0.8029
model_H0 is favored based on adjusted R-squared.


In [181]:
# Leave-One-Out Cross Validation (LOOCV)
h_ii_H0 = model_H0.get_influence().hat_matrix_diag
h_ii_H03 = model_H03.get_influence().hat_matrix_diag

loocv_error_H0 = np.mean(((data["netrent"] - model_H0.fittedvalues) / (1 - h_ii_H0))**2)
loocv_error_H03 = np.mean(((data["netrent"] - model_H03.fittedvalues) / (1 - h_ii_H03))**2)

print(f"LOOCV Error for model_H0: {loocv_error_H0:.4f}")
print(f"LOOCV Error for model H03: {loocv_error_H03:.4f}")

if loocv_error_H03 < loocv_error_H0:
    print("model_H03 is favored based on lower cross-validation error.")
else:
    print("model_H0 is favored based on lower cross-validation error.")

LOOCV Error for model_H0: 4346.6111
LOOCV Error for model H03: inf
model_H0 is favored based on lower cross-validation error.


In [ ]:
# !!!!!!!!!!!!!!!! PROBLEM !!!!!! : inf LOOCV : h_i_i == 1

In [184]:
# Leverage values (hat matrix diagonal elements)
influence_model_H03 = model_H03.get_influence()
h_ii_H03 = influence_model_H03.hat_matrix_diag

# Identify problematic observations where leverage = 1
high_leverage_H03 = np.where(h_ii_H03 == 1)[0]

print("Observations with h_ii == 1 in full model:", high_leverage_H03)

Observations with h_ii == 1 in full model: [2170]


In [188]:
print(len(y))

2371


In [183]:
# Compare AIC
aic_model_H0 = model_H0.aic
aic_model_H03 = model_H03.aic

print(f"AIC for model_H0: {aic_model_H0:.2f}")
print(f"AIC for model_H03: {aic_model_H03:.2f}")

if aic_model_H03 < aic_model_H0:
    print("model_H03 is favored based on AIC.")
else:
    print("model_H0 is favored based on AIC.")

# Compare BIC
bic_model_H0 = model_H0.bic
bic_model_H03 = model_H03.bic

print(f"BIC for model_H0: {bic_model_H0:.2f}")
print(f"BIC for model_H02: {bic_model_H03:.2f}")

if bic_model_H03 < bic_model_H0:
    print("model_H03 is favored based on BIC.")
else:
    print("model_H0 is favored based on BIC.")

AIC for model_H0: 25187.58
AIC for model_H03: 28881.64
model_H0 is favored based on AIC.
BIC for model_H0: 25401.11
BIC for model_H02: 29025.91
model_H0 is favored based on BIC.


# MODEL DIAGNOSIS